<a href="https://colab.research.google.com/github/sahaja-kanuri/Transformer_XL/blob/main/Transformer_XL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initialising

In [ ]:
import tensorflow as tf
import numpy as np
import keras

from keras.models import Model
from keras.layers import Input, Dense, Dropout, LayerNormalization, Embedding
from keras.layers import MultiHeadAttention, Attention, Softmax, Activation, Add
from keras.optimizers import Adam
from keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

In [ ]:
!pip install einops
from einops import rearrange #, repeat, pack, unpack, einsum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.2 MB/s eta 0:00:00


In [ ]:
# Install pre-requisites:
!pip install datasets
from datasets import load_dataset

!pip install sacremoses
from transformers import TransfoXLTokenizer #, TFTransfoXLModel, TFTransfoXLLMHeadModel
import os
os.environ["TRUST_REMOTE_CODE"] = "True"

# Load the Dataset:
wikitext103 = load_dataset('wikitext', 'wikitext-103-v1')
print('The wikitext103 dataset: {wikitext103}')

train_wikitext103 = wikitext103['train']['text']
validation_wikitext103 = wikitext103['validation']['text']
test_wikitext103 = wikitext103['test']['text']
print("\n No. of training examples: ", len(train_wikitext103))
print("\n No. of validation examples: ", len(validation_wikitext103))
print("\n No. of test examples: ", len(test_wikitext103))

# Load the pre-trained tokenizer:
tokenizer = TransfoXLTokenizer.from_pretrained('transfo-xl/transfo-xl-wt103',
                #revision = '40a186da79458c9f9de846edfaea79c412137f97')
                padding_side='right', truncation_side='right',
                pad_token='pad_token')
vocab_size = len(tokenizer)
print(f'Vocab size is: {vocab_size}')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 7.3 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

...

vocab.bin:   0%|          | 0.00/9.14M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/856 [00:00<?, ?B/s]

`TransfoXL` was deprecated due to security issues linked to `pickle.load` in `TransfoXLTokenizer`. See more details on this model's documentation page: `https://github.com/huggingface/transformers/blob/main/docs/source/en/model_doc/transfo-xl.md`.


Vocab size is: 267736


In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
# Initializing Transformer XL model dimensions:
max_sequence_length = 25 #320 #512 # 20 intead 640
train_batch_size = 20 # 32 # 6 # no. of sequences to process at once
validation_batch_size = 32 #64
num_heads = 4 #10 #2
head_dim = 8 #41 # 8
embed_dim = 32 # 410 # 16 # = 64 # num_heads * head_dim
max_iters = 50 #500 #5000
eval_iters = 200
learning_rate = 1e-3
num_layers = 16 #4
num_buckets = 6

edited_tokenized_text currently only contains the first 'train_batch_size' number of sequences, i.e. from index 0 to 'train_batch_size'

# Getting Batches

In [ ]:
# Generalising the above to produce x and y matrices for more than one sequence
# i.e. for 'train_batch_size' number of sequences:

# Generalising the above to produce x and y matrices for more than one sequence
# i.e. for 'train_batch_size' number of sequences:

def get_batch(split, max_sequence_length, train_batch_size):
    '''
    Generates a batch of training/validation/test examples by selecting
    'train_batch_size' number of sequences at random of length
    'max_sequence_length'

    Arguments:
    split -- 'train' or 'validation' or 'test'
    max_sequence_length -- maximum length of the sequence
    train_batch_size -- number of sequences in a batch

    Outputs:
    x -- (train_batch_size, max_sequence_length)
    y -- (train_batch_size, max_sequence_length)
    '''

    # We've picked a random number 32 for now to start by working with only 64 #or 32
    # sequences instead of the entire corpus of wikitext103
    if split == 'train':
        data = train_wikitext103[:128]
    elif split == 'validation':
        data = validation_wikitext103[:128]
    elif split == 'test':
        data = test_wikitext103[:128]

    # Tokenizes the selected data:
    tokenized_text_trial = tokenizer(data,
                                 #, is_split_into_words=False,
                                 truncation=True, max_length=max_sequence_length+1,
                                 padding='max_length',
                                 #padding= True,
                                 return_attention_mask=True,
                                 return_overflowing_tokens=True,
                                 return_tensors='tf')

    # The output tokenized text consists of 0s at the positions where the text
    # has been padded to 'max_sequence_length+1':
    input_attention_mask = tokenized_text_trial['attention_mask']
    tokenized_text = tokenized_text_trial['input_ids']
    #shape: (64, max_sequence_length+1)
    padded_tokenized_text = tf.cast(tf.multiply(tokenized_text,
                                    input_attention_mask), dtype=tf.float32)
    #shape: (64, max_sequence_length+1)

    # Creates an array of random numbers of shape (train_batch_size,)
    # between 0 (incl.) 64 (excl.) #and not max_sequence_length-1 (excl.)
    ix = tf.random.uniform(shape=(train_batch_size,), minval=0,
                           maxval=64, dtype=tf.int32)
                           #maxval=max_sequence_length-1, dtype=tf.int32)
    #print(f'ix: {ix}')
    # x = tf.stack(edited_tokenized_text[i, :max_sequence_length] for i in ix)
    # y = tf.stack(edited_tokenized_text[i, 1:max_sequence_length+1] for i in ix)

    # Pick only the rows of the 1st argument given in the 2nd argument, ix
    # and off-sets the columns by 1 in y compared to x:
    x = tf.gather(padded_tokenized_text[:, :max_sequence_length], ix) #shape: (train_batch_size, max_sequence_length)
    y = tf.gather(padded_tokenized_text[:, 1:max_sequence_length+1], ix) #shape: (train_batch_size, max_sequence_length)

    return x, y #shape: (train_batch_size, max_sequence_length)

In [ ]:
x_trial_train, y_trial_train = get_batch('train', max_sequence_length=max_sequence_length,
                             train_batch_size=train_batch_size)
#train_batch_size= 6, max_sequence_length= 20
# print(f'x_trial: {x_trial}')
# print(f'y_trial: {y_trial}')
# print(x_trial.shape, y_trial.shape)
x_trial_valid, y_trial_valid = get_batch('validation', max_sequence_length=max_sequence_length,
                             train_batch_size=train_batch_size)

In [ ]:
print(x_trial_train.shape)

(20, 25)


In [ ]:
print(x_trial_train)

tf.Tensor(
[[0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00]
 [1.3000e+01 8.2000e+01 5.8000e+01 4.7300e+02 5.1900e+02 2.3400e+02
  2.0000e+00 1.0000e+00 2.4000e+01 2.3400e+02 2.0000e+00 2.3000e+01
  8.7000e+02 7.2000e+01 1.2740e+03 2.6000e+01 2.4000e+01 1.1402e+04
  3.0000e+00 2.1300e+02 2.8220e+03 2.0000e+00 4.8200e+02 5.7620e+03
  1.8300e+02]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000

In [ ]:
# Positional Encodings: DITCH LATER

def get_angles(pos, k, d):
    '''
    Get the angles for the positional encoding

    Arguments:
        pos -- Column vector containing the positions [[0], [1], ...,[N-1]],
        k --   Row vector containing the dimension span [[0, 1, 2, ..., d-1]],
        d(integer) -- Encoding size

    Returns:
        angles -- (pos, d) numpy array
    '''

    # Get i from dimension span k\n",
    i = k//2
    # Calculate the angles using pos, i and d,
    angles = pos/(10000**(2*i/d))

    return angles

def positional_encoding(positions, d):
    '''
    Precomputes a matrix with all the positional encodings

    Arguments:
        positions (int) -- Maximum number of positions to be encoded, #=max_sequence_length
        d (int) -- Encoding size #=embed_dim?

    Returns:
        pos_encoding -- (1, position, d_model) A matrix with the positional encodings
        (d_model = d = embed_dim)
    '''

    # initialize a matrix angle_rads of all the angles \n",
    angle_rads = get_angles(np.array([[pos] for pos in range(positions)]),
                                np.array([[i for i in range(d)]]),
                                d)
    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # apply cos to odd indices in the array; 2i+1\n",
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]
    #pos_encoding = angle_rads # tried, didn't work

    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
pos_encoding = positional_encoding(50, 512)
print(pos_encoding.shape)

(1, 50, 512)


In [ ]:
pos_encoding = positional_encoding(max_sequence_length, embed_dim)
print(pos_encoding.shape)

(1, 25, 32)


**Relative Positional Embedding**

- Positional embeddings are added to the QK embeddings during attention
- Relative position embeddings identify, for each input example, how far away all the other tokens are from a specific token of interest
- Instead of giving each token a relative position index of n that is n positions away from our token of interest, T5 relative position "buckets" some tokens into the same index
- First we create this set of indices. then the indices are matched to an embedding layer of weight values. These values are then added to the QK embeddings during attention. The positional embeddings are trained with the network.


One sentence that is 'max_sequence_length' tokens long which gets split into 'max_sequence_length' number of examples ^

# Relative Positional Encoding layer class

In [ ]:
#1.Construct a relative position matrix
#2.For offsets larger than what we want, start to spread offset values logarithmically into a finite amount of buckets. (Past a certian max value (128) we'll just map everything to one value)
#3.Initialize embedding weights that we will assign offset values to
#4.Now the relative position matrix is mapped to these weights
#5.This matrix gets added to our attention when we perform self-attention. Our self-attention now incorporates as a piece of information the relative positions between tokens

# Relative Positional Encoding:

class relative_positional_encoding_xl(tf.keras.layers.Layer):
    '''
    Constructs a relative position matrix *for one head* to be added to the Attention score

    Arguments:
        rp_scale -- a normalising scale factor
        num_buckets -- default value
        num_heads -- default value = 1
        rp_max_distance -- default value
        sequence_length -- query length/ input sequence length,
                           default value = max_sequence_length

    Returns:
        relative_position_values * self.scale -- shape: (1, num_heads, sequence_length, 2*sequence_length)
    '''
    def __init__(self, rp_scale, num_buckets=num_buckets, num_heads=1,
                 rp_max_distance=max_sequence_length):
        super().__init__()
        # rp stands for relative position
        self.scale = rp_scale
        self.num_buckets = num_buckets
        self.num_heads = num_heads #=1 for single head of attention, later we concatenate for MHA
        self.rp_max_distance = rp_max_distance
        self.relative_position_embedding = Embedding(self.num_buckets, self.num_heads)

    def relative_position_bucket(self, relative_position_matrix):

        # For a decoder model, masking upper triangle after inverting sign:
        n = -relative_position_matrix
        n = tf.math.maximum(n, tf.zeros_like(n))

        # T5 modifications: half of the buckets are for exact increments in position
        max_exact = self.num_buckets//2

        # MASK 1:
        is_small = n < max_exact

        # MASK 2:
        val_if_large = max_exact + (tf.math.log(tf.cast(n, tf.float32) / max_exact) / tf.math.log(self.rp_max_distance / max_exact) * (self.num_buckets - max_exact) )
        val_if_large = tf.cast(val_if_large, dtype=tf.int32)
        val_if_large = tf.minimum(val_if_large, tf.fill(tf.shape(val_if_large), self.num_buckets - 1))

        return tf.where(is_small, n, val_if_large)

    def call(self, sequence_length):
        sequence_pos = tf.range(sequence_length)
        sequence_pos = tf.reshape(sequence_pos, [sequence_pos.shape[0], 1])
        context_pos = tf.range(2*sequence_length)
        #context_pos = tf.range((-sequence_length, sequence_length)
        rel_pos = context_pos - sequence_pos

         # There is no logarithm scaling on the first three, i.e, it stays 3 2 1 0:
        rel_pos_indices = self.relative_position_bucket(rel_pos)

        # We use the above matrix to index into a positional embedding matrix
        # which we initialise randomly to be trained by the model
        # which then gets added to the Attention Scores.
        # There is one of these for each head
        rel_pos_values = self.relative_position_embedding(rel_pos_indices)

        # Need to reshape from (sequence, context, heads) -> (batch, heads, sequence, context):
        rel_pos_values = rearrange(rel_pos_values, 'i j h -> () h i j')
        #equivalently:
        #rel_pos_values = tf.expand_dims(tf.transpose(rel_pos_values, perm=[2,0,1]), axis=0)

        return rel_pos_values * self.scale

In [ ]:
test_relative_positional_xl_encoding = relative_positional_encoding_xl(0.5)
test_call_xl = test_relative_positional_xl_encoding.call(10)
test_call_xl.shape #(1, 1, sequence_length, max_context_length)

TensorShape([1, 1, 10, 20])

In [ ]:
test_relative_positional_xl_encoding.weights
# shape: (num_buckets, num_heads) -- because of the Embedding Layer

[<tf.Variable 'embedding/embeddings:0' shape=(6, 1) dtype=float32, numpy=
 array([[ 0.02681471],
        [-0.02472544],
        [-0.00573098],
        [ 0.03540153],
        [ 0.00726812],
        [ 0.03990949]], dtype=float32)>]

In [ ]:
test_relative_positional_xl_encoding.trainable_weights

[<tf.Variable 'embedding/embeddings:0' shape=(6, 1) dtype=float32, numpy=
 array([[ 0.02681471],
        [-0.02472544],
        [-0.00573098],
        [ 0.03540153],
        [ 0.00726812],
        [ 0.03990949]], dtype=float32)>]

In [ ]:
test_relative_positional_xl_encoding.get_weights()

[array([[ 0.02681471],
        [-0.02472544],
        [-0.00573098],
        [ 0.03540153],
        [ 0.00726812],
        [ 0.03990949]], dtype=float32)]

In [ ]:
# # Estimating the loss: MIGHT DELETE LATER
# def estimate_loss(model, max_sequence_length, train_batch_size):

#     out = {}
#     outputs = model(inputs, training=False)

#     for split in ['train', 'val']:
#         losses = []
#         for k in range(eval_iters):
#             X, Y = get_batch(split, max_sequence_length = max_sequence_length,
#                              train_batch_size=train_batch_size)
#             logits = model(X, training=False)  # Set training to False for inference mode
#             loss = SparseCategoricalCrossentropy(Y, logits)  # Example loss calculation
#             losses.append(loss.numpy())

#         out[split] = tf.reduce_mean(losses)

#     return out

# Attention layer class

In [ ]:
class Attention_Head_xl(tf.keras.layers.Layer):
    '''
    Computes Attention score for one head of self-attention

    Arguments:
        head_size -- Size of each head,
        x -- Input tensor shape: (batch_size, seq_length, embed_dim),
        relative_positions -- shape: (1, num_heads=1, seq_length, 2*seq_length),
        extended_context -- Cached keys and values from prior segment
                of shape: (batch_size, seq_length, 2, head_size)

    Returns:
        output -- shape: (batch_size, seq_length, head_size)
        weights -- shape: (batch_size, num_heads=1, seq_length, 2*seq_length)
        extended_context_cache -- shape: (batch_size, seq_length, 2, head_size)
    '''

    def __init__(self, head_size, dropout_rate=0.1, layernorm_eps=1e-6):

        super().__init__()

        self.head_size = head_size

        self.query = Dense(self.head_size, use_bias=False)
        self.key = Dense(self.head_size, use_bias=False)
        self.value = Dense(self.head_size, use_bias=False)

        # self.mask = self.add_weight(
        #     name='mask',
        #     shape=(seq_length, 2*seq_length),
        #     initializer=tf.keras.initializers.Constant(
        #         tf.linalg.band_part(tf.ones((seq_length, 2*seq_length)), 0, seq_length-1)),
        #     trainable=False)

        self.layer_norm_q = LayerNormalization(epsilon=layernorm_eps)
        self.layer_norm_k = LayerNormalization(epsilon=layernorm_eps)
        self.dropout = Dropout(dropout_rate)


    def call(self, x, relative_positions=None, extended_context=None,
             training=False):
        # maybe an unnecesaary step since these are predefined variables:
        #batch_size, seq_length, head_size = x.shape # embed_dim = x.shape[2], not head size -- check again and again

        # Local Attention:
        q = self.query(x) #(batch_size, seq_length, head_size)
        k = self.key(x) #(batch_size, seq_length, head_size)
        v = self.value(x) #(batch_size, seq_length, head_size)
        # print(f'Attention Head beginning, q shape: {q.shape}')
        # print(f'Attention Head beginning, k shape: {k.shape}')
        # print(f'Attention Head beginning, v shape: {v.shape}')

        # to mitigate model drift/ covariant shift:
        q = self.layer_norm_q(q)
        k = self.layer_norm_k(k)

        if extended_context is not None:
            # Unpack extended_context and concatentate with keys and values
            xl_keys, xl_values = tf.unstack(extended_context, axis=-2)
            # shape: (batch_size, seq_length, head_size)

            xl_seq_length = xl_keys.shape[1]

            # Prepend k and v values along the 'seq_length' axis:
            k = tf.concat([xl_keys, k], axis=-2)
            v = tf.concat([xl_values, v], axis=-2)
            # shape: (batch_size, 2*seq_length, head_size)

        # Compute the dot product of attention scores:
        weights = tf.linalg.matmul(q, k, transpose_b=True) * self.head_size**-0.5
        # if extended_context is not None:
        # shape: (batch_size, seq_length, head_size) * (batch_size, head_size, 2*seq_length)
        # = (batch_size, seq_length, 2*seq_length)
        # the above transposes only the two last axes of the second given tensor

        # weights = tf.expand_dims(weights, axis=1) # weights for each head (axis=1)
        # #shape: = (batch_size, num_heads=1, seq_length, 2*seq_length)
        # print(f'Attention Head, weights expand dims shape: {weights.shape}')

        i, j = weights.shape[-2:] # i = seq_length, j = 2*seq_length


        # Adding relative positional encodings to the attention weights:
        if relative_positions is not None:
            weights = tf.expand_dims(weights, axis=1) # weights for each head (axis=1)
            #shape: = (batch_size, num_heads=1, seq_length, 2*seq_length)
            # print(f'Attention Head, weights expand dims shape: {weights.shape}')

            weights += relative_positions[..., -i:, -j:] # shape of rel_pos: (1, num_heads=1, seq_length, 2*seq_length)
            # shape: (batch_size, num_heads=1, seq_length, 2*seq_length)
            # print(f'Attention Head, weights after rel positions: {weights.shape}')

            weights = tf.squeeze(weights, axis=1)


        # Creates a look-ahead mask (for a causal language model):
        mask = tf.linalg.band_part(tf.ones([i, j], dtype=tf.bool), -1, j-i) == False
        weights = tf.where(mask, tf.fill((i, j), float('-inf')), weights)

        # Perform softmax along the last axis, i.e. along each row/sequence
        weights = Softmax(axis=-1)(weights) # shape: (batch_size, num_heads=1, seq_length, 2*seq_length)
        weights = self.dropout(weights, training=training)

        # weights_temp = tf.squeeze(weights, axis=1)
        # print(f'Attention Head, weights_temp shape: {weights_temp.shape}')
        # shape: (batch_size, seq_length, 2*seq_length)

        # Computes the attention scores ("affinities"):
        # print(f'Attention Head before computing output, weights shape: {weights.shape}')
        # print(f'Attention Head before computing output, v shape: {v.shape}')
        output = tf.linalg.matmul(weights, v)
        # (batch_size, seq_length, 2*seq_length) * (batch_size, 2*seq_length, head_size)
        # = (batch_size, seq_length, head_size)

        # print(f'Attention Head, output shape: {output.shape}')

        # Passing on keys and values:
        kv_memory = tf.stack([k,v], axis=-2)
        # shape: (batch_size, 2*seq_length, 2, head_size), if extended_context is not None
        # But we want (batch_size, seq_length, 2, head_size), which is the case when extended_context is None.
        # For this:

        if extended_context is not None:
            #for all sequences except the first:
            xl_memory, current_input = kv_memory[:, :-xl_seq_length, :, :], kv_memory[:, -xl_seq_length:, :, :]
            extended_context_cache = current_input
            # shape: (batch_size, seq_length, 2, head_size)
            # discard xl_memory
        else:
            #for the first sequence, we don't need to split them out:
            extended_context_cache = kv_memory
            # shape: (batch_size, seq_length, 2, head_size)

        return output, weights, extended_context_cache

In [ ]:
# batch_size = 6
# seq_length = 10
# num_heads = 4
# head_dim = 8
#xl_memory = tf.random.normal([batch_size, seq_length, 2, num_heads*head_dim], dtype=tf.float64)
#xl_memory = np.random.randn(batch_size, seq_length, 2, num_heads*head_dim)
seq_length = max_sequence_length #=20
xl_memory = tf.random.normal([train_batch_size, seq_length, 2, head_dim], dtype=tf.float32)

xl_keys, xl_values = tf.unstack(xl_memory, axis=-2)
print(xl_keys.shape)

(20, 25, 8)


In [ ]:

B, T, C = train_batch_size, max_sequence_length, embed_dim #batch, time, channels (embed_dim not head_size I think)
x_test = tf.random.normal([B,T,C]) #, dtype=tf.float32) # shape: (B, T, C)
context_test = tf.random.normal([B, T, 2, head_dim]) #, dtype=tf.float32)


In [ ]:

rel_pos = relative_positional_encoding_xl(0.5)
rel_pos = rel_pos.call(seq_length)
# shape: (1, num_heads=1, seq_length, 2*seq_length)

attention_call_xl = Attention_Head_xl(head_size=head_dim)
output_call, weights_call, extended_out_call = attention_call_xl(x_test, relative_positions= rel_pos, training=True) # extended_context=context_test
#, weights_call

In [ ]:
print(output_call.shape)
print(extended_out_call.shape)

(20, 25, 8)
(20, 25, 2, 8)


In [ ]:
print(x_test.shape)

(20, 25, 32)


In [ ]:
print(weights_call.shape)
print(output_call.shape)
print((extended_out_call.shape))

(20, 25, 25)
(20, 25, 8)
(20, 25, 2, 8)


In [ ]:
# attention_call_xl.trainable_weights

# Multi-Head Attention layer class

In [ ]:
class Multi_Head_Attention_xl(tf.keras.layers.Layer):
    '''
    Uses Attention_Head to implement multiple heads of self-attention

    Arguments:
        head_size -- Size of each head (a type of communication channel, C),
        num_heads -- Number of heads,
        x -- Input tensor shape: (batch_size, seq_length, embed_dim)
        relative_positions_mha -- relative positional encoding matrix
        extended_context_mha -- defaults to None, or of shape: (batch_size, seq_length, 2, head_size * num_heads)

    Returns:
        out -- shape: (batch_size, seq_length, head_size * num_heads=embed_dim),
        where head_size * num_heads = embed_dim ,
        extended_context_mha -- shape: (batch_size, seq_length, 2, head_size)
    '''
    def __init__(self, head_size, num_heads, dropout_rate=0.1):
        super().__init__()
        #self.head1 = Attention_Head_xl(head_size)
        self.head_size = head_size
        self.num_heads = num_heads
        self.embed_dim = self.head_size * self.num_heads
        # assert self.embed_dim % self.num_heads == 0

        self.heads = [Attention_Head_xl(self.head_size) for _ in range(self.num_heads)]
        self.head = Attention_Head_xl(self.head_size)
        self.projection = Dense(self.embed_dim)
        self.dropout = Dropout(dropout_rate)

    def call(self, x, relative_positions_mha=None, extended_context_mha=None, training=False):
        '''
        training: boolean, i.e. either 'True' or 'False'
        Let extended_context_mha stay None when initialising, as for the
        first sequence, there will be no prior context.
        '''
        #out, _ , self_attention_context = self.head1(x)
        out = []
        #weights_out = []
        '''
        # iter = 0
        for i, head in enumerate(self.heads):
            # print(f'Iter no.: {iter}')
            print(f'i = {i}')
            output, _, extended_context_mha = head(x,
                                                    relative_positions=relative_positions_mha,
                                                    extended_context=extended_context_mha,
                                                    training=training)
            # print(output.shape)

            # shape of output: (batch_size, seq_length, head_size)
            out.append(output)
            #weights_out.append(weights)
        # print(f'Out shape: {len(out)}')
        '''
        '''
        for i in range(self.num_heads):
            output, _, extended_context_mha = self.head(x,
                                                    relative_positions=relative_positions_mha,
                                                    extended_context=extended_context_mha,
                                                    training=training)
            out.append(output)
        '''
        for i in range(self.num_heads):
            # pass x and the encoder output through a stack of decoder layers and
            # save the attention weights of block 1 and 2
            output, _, extended_context_mha = self.heads[i](x,
                                                    relative_positions=relative_positions_mha,
                                                    extended_context=extended_context_mha,
                                                    training=training)
            out.append(output)
            # print(f'1. Output shape: {output.shape}')

        out = tf.concat(out, axis=-1)
        # print(f'2. Out shape: {out.shape}')
        # shape: (batch_size, seq_length, head_size * num_heads=embed_dim)

        #assert out.shape[2] == x.shape[2]

        out = self.projection(out)
        out = self.dropout(out, training=training)
        # print(f'3. Out shape: {out.shape}')
        #weights_out = tf.concat(weights_out, axis=1)

        return out, extended_context_mha

In [ ]:
rel_pos = relative_positional_encoding_xl(0.5)
rel_pos = rel_pos.call(seq_length)


multi_head_attention_xl = Multi_Head_Attention_xl(head_size=head_dim, num_heads=num_heads)
multi_head_attention_output, extended_context_mha_out = multi_head_attention_xl(x_test, relative_positions_mha=rel_pos, extended_context_mha=context_test, training=True) #relative_positions_mha=rel_pos, extended_context_mha=context_test

In [ ]:
print(multi_head_attention_output.shape)
print(extended_context_mha_out.shape)
print(x_test.shape)

(20, 25, 32)
(20, 25, 2, 8)
(20, 25, 32)


# Feed Forward Neural Network layer class

In [ ]:
class FeedForward(tf.keras.layers.Layer):
    '''
    Implements a feed-forward network

    Arguments:
        embed_dim -- Size of the input embedding
        x -- Input tensor shape: (batch_size, seq_length, head_size * num_heads=embed_dim)

    Returns:
        output -- shape: (batch_size, seq_length, embed_dim)
    '''
    def __init__(self, embed_dim, dropout_rate=0.1):
        super().__init__()
        full_connected_dim = 4 * embed_dim
        self.ffn = tf.keras.Sequential([
                    Dense(full_connected_dim, activation='relu'),
                    Dropout(dropout_rate),
                    Dense(embed_dim),
                    Dropout(dropout_rate)
                    ])
        # self.dropout = Dropout(dropout_rate)
    def call(self, x, training=False):
        # out = self.ffn(x)
        # out = self.dropout(out, training=training)
        #return out
        return self.ffn(x, training=training)

In [ ]:
ffn_trial = FeedForward(embed_dim=embed_dim)
ffn_trial_output = ffn_trial(x_test, training=True)
print(ffn_trial_output.shape)

(20, 25, 32)


In [ ]:
ffn_trial.trainable_weights

[<tf.Variable 'dense_2/kernel:0' shape=(32, 128) dtype=float32, numpy=
 array([[-0.13023561,  0.01492491, -0.10715895, ..., -0.12327716,
          0.04630895,  0.1727846 ],
        [ 0.02921878,  0.04240493,  0.0412827 , ...,  0.04743429,
          0.09207729, -0.04176337],
        [ 0.10089651,  0.06869057, -0.18197916, ...,  0.15648183,
         -0.12050999,  0.15378442],
        ...,
        [ 0.02102543, -0.14897025,  0.08854374, ...,  0.08053619,
         -0.031132  ,  0.1439752 ],
        [-0.01513392,  0.07650971, -0.10071959, ...,  0.17248085,
          0.19027165, -0.01965263],
        [-0.16046028,  0.04198807, -0.07286535, ..., -0.13382995,
         -0.1509981 , -0.04817253]], dtype=float32)>,
 <tf.Variable 'dense_2/bias:0' shape=(128,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

# Transformer Decoder layer class

In [ ]:
class Transformer_xl_Decoder(tf.keras.layers.Layer):
    '''
    Implements a transformer decoder block

    Arguments:
        embed_dim -- Size of the input embedding
        num_heads -- Number of heads
        x -- Input tensor shape: (batch_size, seq_length, embed_dim)

    Returns:
        output -- shape: (batch_size, seq_length, embed_dim)
    '''
    def __init__(self, num_heads, head_size, layernorm_eps=1e-6):
        super().__init__()
        #head_size = embed_dim // num_heads
        self.embed_dim = num_heads * head_size
        #self.token_embedding = TokenEmbedding(vocab_size, embed_dim)
        #self.positional_encoding = PositionalEncoding(seq_length, embed_dim)
        self.Multi_Head_Attention = Multi_Head_Attention_xl(head_size, num_heads)
        self.FeedForward = FeedForward(self.embed_dim)
        self.layernorm1 = LayerNormalization(epsilon=layernorm_eps)
        self.layernorm2 = LayerNormalization(epsilon=layernorm_eps)

    def call(self, x, rel_pos_decoder=None, ext_context=None, training=False):
        #x = self.token_embedding(x)
        #x = self.positional_encoding(x)
        # Skip connection/ residual connection,
        # layer norm is applied before mha, a slight deviation from
        # the original Attention paper:

        residual = tf.identity(x)
        #residual = tf.expand_dims(x, axis=-1)
        # shape of residual: (batch_size, seq_length)

        x, ext_context = self.Multi_Head_Attention(self.layernorm1(x),
                                                relative_positions_mha=rel_pos_decoder,
                                                extended_context_mha=ext_context,
                                                training=training)
        #shape of x: (batch_size, seq_length, head_size * num_heads)

        # print(f'Residual shape: {residual.shape}')
        # print(f'x shape: {x.shape}')
        x = Add()([residual, x])


        # x = Add()([x, self.Multi_Head_Attention(self.layernorm1(x),
        #                                         relative_positions_mha=rel_pos,
        #                                         extended_context_mha=ext_context,
        #                                         training=training)[0]])
        # Skip connection/ residual connection:
        x = Add()([x, self.FeedForward(self.layernorm2(x), training=training)])
        #shape: (batch_size, seq_length, embed_dim)
        return x, ext_context

In [ ]:
rel_pos = relative_positional_encoding_xl(0.5)
rel_pos = rel_pos.call(seq_length)
print(rel_pos.shape) #(1, num_heads=1, sequence_length, 2*sequence_length)

transformer_trial = Transformer_xl_Decoder(num_heads=num_heads, head_size=head_dim)
transformer_trial_output, _ = transformer_trial(x_test, rel_pos_decoder=rel_pos, training=True)
print(transformer_trial_output.shape)

(1, 1, 25, 50)
(20, 25, 32)


In [ ]:
# test_relative_positional_encoding = relative_positional_encoding_xl(0.5)
# test_call = test_relative_positional_encoding.call(seq_length)
# test_call.shape #(1, num_heads, sequence_length, max_context_length)

# Transformer XL model class

In [ ]:
class Transformer_xl(tf.keras.models.Model):
    '''
    A Model class consisting of all trainable layers
    '''
    def __init__(self, num_layers, embed_dim, num_heads, seq_length,
                rp_scale, dropout_rate=0.1, layernorm_eps=1e-6):
        super().__init__()

        self.num_layers = num_layers
        self.embed_dim = embed_dim
        self.seq_length = seq_length
        self.num_heads = num_heads
        self.head_size = embed_dim//num_heads

        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = Embedding(vocab_size, self.embed_dim)
        self.position_embedding_table = Embedding(self.seq_length, self.embed_dim)

        # Generates the relative positional encoding matrix
        self.rel_pos_enc = relative_positional_encoding_xl(rp_scale)

        self.decoder_layers = [Transformer_xl_Decoder(num_heads= self.num_heads,
                                                      head_size = self.head_size)
                                for _ in range(self.num_layers)]
        self.layernorm = LayerNormalization(epsilon=layernorm_eps) #final layer norm
        self.head = Dense(vocab_size)

    def call(self, idx, ext_context=None, training=False):
        B, T = idx.shape # B= batch_size, T = seq_length

        tok_emb = self.token_embedding_table(idx) # (B,T,embed_dim)
        pos_emb = self.position_embedding_table(tf.range(T)) # (T,embed_dim)
        x = tok_emb + pos_emb #  (B,T,embed_dim)

        # print(f'Residual model shape: {x.shape}')

        rel_pos = self.rel_pos_enc.call(T)
        # shape: (1, num_heads, T, 2*T)

        for i in range(self.num_layers):
            # pass x and the encoder output through a stack of decoder layers and
            # save the attention weights of block 1 and 2
            x, ext_context = self.decoder_layers[i](x, rel_pos_decoder=rel_pos,
                                                    ext_context=ext_context, training=training)
            # (B,T,embed_dim)

        x = self.layernorm(x) # (B,T,embed_dim)
        logits = self.head(x) ## (B,T,vocab_size)

        # B, T, C = logits.shape
        # logits = logits.view(B*T, C)
        # targets = targets.view(B*T)
        # loss = CategoricalCrossentropy()(targets, logits)

        return logits

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -seq_length:]
            # get the predictions
            logits = self(idx_cond) #logits, loss previously
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = Softmax(axis=-1)(logits) # (B, C)
            # sample from the distribution
            idx_next = tf.random.categorical(logits=probs,  num_samples=1) # (B, 1)
            idx_next = tf.cast(idx_next, tf.int32)
            #input to tf.concat must be of type int32

            # append sampled index to the running sequence
            idx = tf.concat([idx, idx_next], axis=1) # (B, T+1)

        return idx

# Training

In [ ]:
model = Transformer_xl(num_layers=num_layers, embed_dim=embed_dim, num_heads=num_heads,
                       seq_length=max_sequence_length, rp_scale = 0.5)

In [ ]:
optimizer_use = Adam(learning_rate=learning_rate)

In [ ]:
# max_iters = 5000
eval_interval = 4
for iter in range(max_iters):
    # every once in a while evaluate the loss on train and val sets
    # if iter % eval_interval == 0 or iter == max_iters - 1:
    #     losses = estimate_loss(model2, max_sequence_length=max_sequence_length,
    #                            train_batch_size=train_batch_size)
    #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train', max_sequence_length=max_sequence_length,
                       train_batch_size=train_batch_size)

    # evaluate the loss
    model.compile(
        optimizer='adam',
        loss=SparseCategoricalCrossentropy(),
        metrics=['accuracy']
        )

    print(f'{iter+1}. Training:')
    model.fit(xb, yb)

    xb_valid, yb_valid = get_batch('validation', max_sequence_length=max_sequence_length,
                       train_batch_size=train_batch_size)

    print(f'Validation:')
    loss_validation = model.evaluate(xb_valid, yb_valid)
    #print(f'{iter}. validation loss: {loss_validation}')

1. Training:
1/1 [==============================] - 182s 182s/step - loss: 19.4279 - accuracy: 0.0000e+00
Validation:
1/1 [==============================] - 12s 12s/step - loss: 14.6494 - accuracy: 0.0020
2. Training:
1/1 [==============================] - 156s 156s/step - loss: 11.3239 - accuracy: 0.1020
Validation:
1/1 [==============================] - 12s 12s/step - loss: 9.5439 - accuracy: 0.0260
3. Training:
1/1 [==============================] - 155s 155s/step - loss: 7.3704 - accuracy: 0.0720
Validation:
1/1 [==============================] - 11s 11s/step - loss: 7.7412 - accuracy: 0.4300
4. Training:
1/1 [==============================] - 165s 165s/step - loss: 5.2619 - accuracy: 0.6040
Validation:
1/1 [==============================] - 15s 15s/step - loss: 7.1587 - accuracy: 0.5000
5. Training:


In [ ]:
context = tf.zeros((1, 1), dtype=tf.int32)
prediction = model.predict(context)

In [ ]:
print(prediction.shape)

In [ ]:
# generate from the model
context = tf.zeros((1, 1), dtype=tf.int32)
generated_tokens = model.generate(context, max_new_tokens=20)
print(tokenizer.decode(generated_tokens[0].numpy().tolist(), skip_special_tokens=True))